# Tools

## Calculator example

In this example, the docstring and inferred arguments and argument types are used by the LLM to detetermine when and how to call the tool.

In [1]:
from typing import Literal

from langchain.tools import tool


@tool
def real_number_calculator(
    a: float, b: float, operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """Perform basic arithmetic operations on two real numbers."""
    print("Invoking calculator tool")
    # Perform the specified operation
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Division by zero is not allowed.")
        return a / b
    else:
        raise ValueError(f"Invalid operation: {operation}")

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[real_number_calculator],
    system_prompt="You are a helpful assistant",
)

This invokes your calculator tool.

In [3]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "what is 3.1125 * 4.1234"}]}
)
print(result["messages"][-1].content)

3.1125 × 4.1234 = 12.8340825


We can check the **metadata** in [LangSmith Observability](https://smith.langchain.com/public/b77bde6c-f0ad-4256-bfab-7d514ece3405/r) to see this.

The tool description can have a big impact. 
This may not  invoke your calculator tool because the inputs are integers.  (results vary from run to run)

In [4]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3 * 4"}]})
print(result["messages"][-1].content)

Invoking calculator tool
12


This often does not invoke the tool though the input are real numbers

In [5]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3.0 * 4.0"}]})
print(result["messages"][-1].content)

Invoking calculator tool
12.0


## Adding a more detailed description
While a basic description is often sufficient, LangChain has support for enhanced descriptions. The example below uses one method: Google Style argument descriptions. Used with `parse_docstring=True`, this will parse and pass the arg descriptions to the model. The `description` can provide a means of replacing the tool description with an Agent-specific description that can include specific instructions on when and how to call the tool.

In [11]:
from typing import Literal

from langchain.tools import tool


@tool(
    parse_docstring=True,
    description=(
        "Perform basic arithmetic operations on two real numbers. Use this whenever you have operations on non-integers."
    ),
)
def real_number_calculator(
    a: float, b: float, operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """Perform basic arithmetic operations on two real numbers.

    Args:
        a (float): The first number.
        b (float): The second number.
        operation (Literal["add", "subtract", "multiply", "divide"]):
            The arithmetic operation to perform.

            - `"add"`: Returns the sum of `a` and `b`.
            - `"subtract"`: Returns the result of `a - b`.
            - `"multiply"`: Returns the product of `a` and `b`.
            - `"divide"`: Returns the result of `a / b`. Raises an error if `b` is zero.

    Returns:
        float: The numerical result of the specified operation.

    Raises:
        ValueError: If an invalid operation is provided or division by zero is attempted.
    """
    print("Invoking calculator tool")
    # Perform the specified operation
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Division by zero is not allowed.")
        return a / b
    else:
        raise ValueError(f"Invalid operation: {operation}")

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[real_number_calculator],
    system_prompt="You are a helpful assistant",
)

In [15]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3.0 * 4.0"}]})
print(result["messages"][-1].content)

Invoking calculator tool
12.0


Let's check our [trace](https://smith.langchain.com/public/5e5c22be-278b-4251-8e70-f0ea9a7e17b5/r) to see the tool description.

In [14]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3 * 4"}]})
print(result["messages"][-1].content)

12
